# Getting started with Spotipy

Start off in a notebook to see if the endpoints give me everything I need

## Connect and authenticate

You'll need to have set up an App in your Spotify dashboard first

In [ ]:
import random
import re

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import yaml


with open("params.yaml", "r") as f:
    params = yaml.safe_load(f)

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=params["client_id"],
        client_secret=params["client_secret"],
        redirect_uri=params["redirect_uri"],
        scope="playlist-modify-private",
    )
)

## Extract playlist IDs

In [ ]:
# Get playlist ID from 'Copy link to playlist'
# ID is the url substring from after '/playlist/' up to the question mark
id_pattern = re.compile("/playlist/(\w+)\?")
input_playlist_url = params["input_playlist"]
input_playlist_id = id_pattern.search(input_playlist_url)[1]
output_playlist_url = params["output_playlist"]
output_playlist_id = id_pattern.search(output_playlist_url)[1]

## Pick songs from the input playlist

Basically this should be one song (or more, to allow weighting) from each album you want to shuffle.

Perhaps an option to shuffle by artist instead? Grab any album of theirs.

In [ ]:
# Use API to get the playlist content
fields = "tracks.items(track(album))"
playlist = sp.playlist(input_playlist_id, fields=fields)
songs = playlist["tracks"]["items"]

# Pick 1 or more songs from it
random.seed = None
pick = 1
chosen_songs = random.choices(songs, k=pick)

## Send a random album to the output playlist

This should complete the minimum process

In [ ]:
for song in chosen_songs:
    album_id = song["track"]["album"]["id"]
    album_tracks = sp.album_tracks(album_id)
    album_track_ids = [track["id"] for track in album_tracks["items"]]
    sp.playlist_add_items(output_playlist_id, album_track_ids)
    # or use playlist_replace_items if I want to delete everything already there?